In [3]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

with open('/content/doc1.txt', 'r', encoding='utf-8') as file:
    doc1 = file.read()

with open('/content/doc2.txt', 'r', encoding='utf-8') as file:
    doc2 = file.read()

documents = [doc1, doc2]

tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(documents)

cos_sim = cosine_similarity(tfidf_matrix[0:1], tfidf_matrix)
print("Cosine Similarity between the two documents:", cos_sim[0, 1])


Cosine Similarity between the two documents: 0.56169019227452


TF-IDF stands for Term Frequency-Inverse Document Frequency. It's a numerical representation of a document (a piece of text) that captures the importance of each word within that document relative to a collection of documents.

Term Frequency (TF): This measures how often a word appears in a document. If a word appears more frequently in a document, its TF value will be higher.

Inverse Document Frequency (IDF): This measures how unique or rare a word is across all the documents in the collection. If a word appears in many documents, its IDF value will be lower.

Cosine similarity is a measure of similarity between two vectors (in this case, TF-IDF vectors). It calculates the cosine of the angle between these vectors, which represents how similar they are in direction.

In [4]:
def jaccard_similarity(doc1, doc2):
    set1 = set(doc1.split())
    set2 = set(doc2.split())
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union

jac_sim = jaccard_similarity(doc1, doc2)
print("Jaccard Similarity between the two documents:", jac_sim)


Jaccard Similarity between the two documents: 0.10454545454545454


Jaccard similarity is a measure used to compare the similarity between two sets. In simple terms, it tells us how similar or different two sets are by considering the intersection (common elements) and union (total elements) of the sets.

Imagine you have two sets, Set A and Set B. The Jaccard similarity between Set A and Set B is calculated by dividing the number of elements they have in common by the total number of unique elements in both sets.

In [10]:
import nltk
nltk.download('stopwords')
import nltk
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

#Baeysian Classification

In [15]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import defaultdict

df = pd.read_csv('/content/Tweets.csv')

dataset = df[['text', 'airline_sentiment']]

stop_words = set(stopwords.words('english'))
word_freq = defaultdict(lambda: [0, 0])
for _, row in dataset.iterrows():
    text = row['text']
    label = row['airline_sentiment']
    words = [word.lower() for word in word_tokenize(text) if word.isalnum() and word.lower() not in stop_words]
    for word in words:
        word_freq[word][label == 'positive'] += 1

total_positive = sum(word_freq[word][1] for word in word_freq)
total_negative = sum(word_freq[word][0] for word in word_freq)
prior_positive = total_positive / (total_positive + total_negative)
prior_negative = total_negative / (total_positive + total_negative)

def classify(text):
    words = [word.lower() for word in word_tokenize(text) if word.isalnum() and word.lower() not in stop_words]
    log_prob_positive = sum([word_freq[word][1] / total_positive for word in words])
    log_prob_negative = sum([word_freq[word][0] / total_negative for word in words])
    prob_positive = prior_positive * log_prob_positive
    prob_negative = prior_negative * log_prob_negative
    return 'positive' if prob_positive > prob_negative else 'negative'

test_data = ["@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing. it's really the only bad thing about flying VA"]
for text in test_data:
    sentiment = classify(text)
    print(f"Sentiment of '{text}': {sentiment}")


Sentiment of '@VirginAmerica seriously would pay $30 a flight for seats that didn't have this playing. it's really the only bad thing about flying VA': negative


HERE we are giving a sample text from the dataste which is the above mentioned, then we are trying to analyse the sentiment in that statemnt.

here in this example the sentiment is negative

Bayesian Classification is a method used in machine learning for predicting the category or class of a given data point based on the probability that it belongs to each category. It's based on Bayes' theorem, which is a fundamental concept in probability theory.



#RNN

In [16]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [17]:
df = pd.read_csv('/content/Tweets.csv')

In [14]:
texts = df['text'].tolist()
labels = df['airline_sentiment'].tolist()

label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

tokenizer = Tokenizer()
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

max_len = max(len(seq) for seq in sequences)
sequences_padded = pad_sequences(sequences, maxlen=max_len, padding='post')

X_train, X_test, y_train, y_test = train_test_split(sequences_padded, labels, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(input_dim=len(tokenizer.word_index)+1, output_dim=32, input_length=max_len))
model.add(LSTM(32))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, np.array(y_train), epochs=10, batch_size=1, verbose=1)

loss, accuracy = model.evaluate(X_test, np.array(y_test))
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)


Epoch 1/10
11712/11712 [==============================] - 228s 19ms/step - loss: -0.1645 - accuracy: 0.6362
Epoch 2/10
11712/11712 [==============================] - 217s 18ms/step - loss: -3.2473 - accuracy: 0.6587
Epoch 3/10
11712/11712 [==============================] - 218s 19ms/step - loss: -2.5985 - accuracy: 0.5503
Epoch 4/10
11712/11712 [==============================] - 207s 18ms/step - loss: -11.9200 - accuracy: 0.6774
Epoch 5/10
11712/11712 [==============================] - 210s 18ms/step - loss: -23.3699 - accuracy: 0.6954
Epoch 6/10
11712/11712 [==============================] - 212s 18ms/step - loss: -35.6627 - accuracy: 0.7123
Epoch 7/10
11712/11712 [==============================] - 213s 18ms/step - loss: -51.1009 - accuracy: 0.7297
Epoch 8/10
11712/11712 [==============================] - 213s 18ms/step - loss: -64.3834 - accuracy: 0.7426
Epoch 9/10
11712/11712 [==============================] - 218s 19ms/step - loss: -83.8500 - accuracy: 0.7526
Epoch 10/10
92/92 [===

as we can see the accuracy of the model is increasing with each epoche.

also the test accuracy is about 69 percent

rnn are helpful in porcessing test data and analysing the snetiment in them

#LSTM

In [20]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import LSTM, Embedding, Dense, SpatialDropout1D
from keras.callbacks import EarlyStopping


In [21]:
data = pd.read_csv("/content/Tweets.csv")

In [22]:
X = data['text']
y = pd.get_dummies(data['airline_sentiment']).values

tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=200)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = Sequential()
model.add(Embedding(5000, 128, input_length=X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, restore_best_weights=True)

history = model.fit(X_train, y_train, epochs=10, batch_size=64, validation_data=(X_test, y_test), callbacks=[early_stop])

loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test Accuracy:', accuracy)


Epoch 1/10
183/183 [==============================] - 160s 855ms/step - loss: 0.7165 - accuracy: 0.7018 - val_loss: 0.5323 - val_accuracy: 0.7889
Epoch 2/10
183/183 [==============================] - 155s 848ms/step - loss: 0.4687 - accuracy: 0.8155 - val_loss: 0.5003 - val_accuracy: 0.8046
Epoch 3/10
183/183 [==============================] - 145s 791ms/step - loss: 0.3733 - accuracy: 0.8613 - val_loss: 0.5163 - val_accuracy: 0.8029
Epoch 4/10
183/183 [==============================] - 144s 787ms/step - loss: 0.3188 - accuracy: 0.8794 - val_loss: 0.5359 - val_accuracy: 0.8026
Epoch 5/10
183/183 [==============================] - 140s 767ms/step - loss: 0.2774 - accuracy: 0.8993 - val_loss: 0.5650 - val_accuracy: 0.7978
Epoch 5: early stopping
Test Accuracy: 0.8046448230743408


for the lstm model we introduce an early stop for epoch to prevent overfitting of the model

the test accuracy is about 89percent